<a href="https://colab.research.google.com/github/PardonMySkillz/PardonMySkillz/blob/main/EE331_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# 코드로 형식 지정됨
```

# 2023 Fall EE331 Project


**OBJECTIVE**

You are provided with a simple PyTorch model that classfies images into one of three categories: rock, paper or scissors. The current model uses just one fully connected (FC) layer for classfication. Your task is to do the following:

1. Model Design


> Replace the single FC layer in the '' class with a neural network architecture of your choice. Consider using combinations of layers such as multiple FC layers, convolutional layers, activation functions and dropout layers.


2. Training


> Once the model is designed,

use the providced training loop to train the model. Adjust the hyperparameters if necessary. You may change the provided dataset with a dataset of your choice. However, this must be stated in the report you are going to write.


You are free to change the code sections 1~????.

For more detail on the report, refer to the project pdf file uploaded in KLMS

0. Libraries

In [1]:
import os
from typing_extensions import dataclass_transform
import pandas as pd
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from torchvision.utils import make_grid
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn.functional as F
import glob
import cv2
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt


# !pip install -Uqq ipdb
# import ipdb
# %pdb on


drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# import zipfile

# zip_ref = zipfile.ZipFile("/content/drive/MyDrive/archive 4.zip", 'r')
# zip_ref.extractall("/content/drive/MyDrive/archive 4")
# zip_ref.close()

**1. DATASET**

The following link has some rock scissors paper dataset you can use:

https://kaistackr-my.sharepoint.com/:f:/g/personal/shimjay17_kaist_ac_kr/EkBKx6r7RU1BnsFc5QbqFBMBHhOzF8SFoHXzaJAr1n7a3g?e=ElWBrF

To use this dataset, you will have to upload these datasets to your google drive and specify its location in the dataset path in the code below.

However, though it is not a necessity, we recommend that you test your model on additional datasets that are available to use. Remember that your model is going to be graded on an unseen custom dataset and will require your model good generalisation.

If you do decide to use an additional dataset, you may need to edit the 'loadLabels' definition accordingly in the 'Dataset' class.

In [4]:

######### Declare dataset path below! #########
DATASET_PATH = '/content/drive/MyDrive/archive 2/Rock-Paper-Scissors/Rock-Paper-Scissors/train'
###############################################

from torchvision import transforms

class Dataset(Dataset):
    def __init__(self, dataset_dir, transform=None, target_transform=None):
        self.class_list = {'rock': 0, 'scissors': 1, 'paper': 2}
        self.labels_map = {0:'rock', 1:'scissors', 2:'paper'}
        self.path = DATASET_PATH
        self.image_paths = sorted(glob.glob(os.path.join(self.path, '**', '*.png'), recursive=True))
        self.transform = transforms.Compose([transforms.ToTensor()]) if transform is None else transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = self.loadImage(img_path)
        image = cv2.resize(image, (28, 28))

        # Apply the transform to the image
        if self.transform:
            image = self.transform(image)

        label = self.loadLabels(img_path)
        return image, torch.tensor(label, dtype=torch.long)  # Ensure label is a long tensor

    def loadImage(self, path):
        img = cv2.imread(path)
        img = img.astype(np.float32)/255.0  # 이미지
        return img

    def loadLabels(self, image_paths):
        class_name = os.path.dirname(image_paths).split('/')[-1]
        return self.class_list[class_name]

**2. Classification Code**

In this section, you are required to implement a network of your choice to get better performance.
You are free to edit this skeleton code however you want.

In [5]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        self.conv1 = nn.Conv2d(3, 6, 5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        print(np.shape(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

**3. Train & Evaluation Code**
Here is an example Train and Evaluation code for the baseline FC layer model. You will most likely have to change the code below to fit your model of choice.

In [6]:
def train(train_dataset, val_dataset, model, device='cuda:0' if torch.cuda.is_available() else 'cpu'):
    model.to(device)
    model.train()

    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)
    learning_rate = 0.001
    epoch_no = 10
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    print("====TRAIN INITIATED====")
    print(model)

    for epoch in range(epoch_no):
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for idx, data in enumerate(train_dataloader):
            imgs, labels = data
            imgs, labels = imgs.to(device), labels.to(device)
            output = model(imgs)

            _, predicted = torch.max(output.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

            model_loss = F.cross_entropy(output, labels)
            running_loss += model_loss.item()
            model_loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        avg_train_loss = running_loss / len(train_dataloader)
        train_accuracy = 100 * correct_train / total_train
        print(f"Epoch {epoch + 1} - Average training loss: {avg_train_loss:.3f}, Training accuracy: {train_accuracy:.2f}%")

        # validation loop every 5 epochs
        if (epoch + 1) % 2 == 0:
            val_loss = 0
            correct_val = 0
            total_val = 0
            with torch.no_grad():
                for val_idx, (val_imgs, val_labels) in enumerate(val_dataloader):
                    val_imgs, val_labels = val_imgs.to(device), val_labels.to(device)
                    val_output = model(val_imgs)

                    _, val_predicted = torch.max(val_output.data, 1)
                    total_val += val_labels.size(0)
                    correct_val += (val_predicted == val_labels).sum().item()

                    val_loss += F.cross_entropy(val_output, val_labels)

                val_accuracy = 100 * correct_val / total_val
                print(f'Validation loss: {val_loss/(val_idx+1):.3f}, Validation accuracy: {val_accuracy:.2f}%')

def test(test_dataset, model, device='cuda:0' if torch.cuda.is_available() else 'cpu'):
    model.to(device)
    model.eval()

    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

    correct_test = 0
    total_test = 0
    test_loss = 0
    with torch.no_grad():
        for idx, (test_imgs, test_labels) in enumerate(test_dataloader):
            test_imgs, test_labels = test_imgs.to(device), test_labels.to(device)
            test_output = model(test_imgs)

            _, predicted = torch.max(test_output.data, 1)
            total_test += test_labels.size(0)
            correct_test += (predicted == test_labels).sum().item()

            test_loss += F.cross_entropy(test_output, test_labels)

        test_accuracy = 100 * correct_test / total_test
        print(f'Test loss: {test_loss/(idx+1):.3f}, Test accuracy: {test_accuracy:.2f}%')


**4. Run Code**

Run the code below to run all the whole model.

In [7]:
if __name__ == '__main__':

    dataset = Dataset(DATASET_PATH)
    dataset_size = len(dataset)
    train_size = int(dataset_size * 0.8)
    validation_size = int(dataset_size * 0.1)
    test_size = dataset_size - train_size - validation_size

    train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])

    print(f"Training Data Size : {len(train_dataset)}")
    print(f"Validation Data Size : {len(validation_dataset)}")
    print(f"Testing Data Size : {len(test_dataset)}")

    model = Classifier()
    train(train_dataset, validation_dataset, model)
    test(test_dataset, model)

    PATH = './WongHoiChun_20236391.pth'
    torch.save(model.state_dict(), PATH)

Training Data Size : 1039
Validation Data Size : 129
Testing Data Size : 131
====TRAIN INITIATED====
Classifier(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=3, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
torch.Size([32, 400])
torch.Size([32, 400])
torch.Size([32, 400])
torch.Size([32, 400])
torch.Size([32, 400])
torch.Size([32, 400])
torch.Size([32, 400])
torch.Size([32, 400])
torch.Size([32, 400])
torch.Size([32, 400])
torch.Size([32, 400])
torch.Size([32, 400])
torch.Size([32, 400])
torch.Size([32, 400])
torch.Size([32, 400])
torch.Size([32, 400])
torch.Size([32, 400])
torch.Size([32, 400])
torch.Size([32, 400])
torch.Size([32, 400])


**5. Inference**

Once you are satisfied with your classifier code, you are to edit the code below so that it runs. The code below should successfully be able to load the 'test_image.jpeg' within the 'Project items' folder in the link given above and give a prediction for the image using your proposed model.

The final output should return either 'rock', 'scissors' or 'paper'.

In [12]:
chosen_img_path = '/content/drive/MyDrive/test_image.jpeg'

################ Edit Here ################



################ Edit Here ################

output = model(input)
print(output)


torch.Size([1, 400])
tensor([0.9036, 0.0913, 0.0052])


TypeError: ignored